In [ ]:
%matplotlib inline
#%pylab
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mpc
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

import scipy
from scipy import interpolate
import pandas as pd
import itertools
from root_numpy import root2array, root2rec, tree2rec, array2root
from ROOT import TChain

In [ ]:
def open_df(fname,treename="flashtrigger/opflash_opflash_tree",start=None,stop=None):
    return pd.DataFrame( root2array(fname, treename,start=start,stop=stop) )

In [ ]:
#
# Data loading
#
rate=5
minutes=0
hour=0
day=1

#num_cosmic = 5*60*(minutes+hour*60+day*60*24)
#num_bnb = int(num_cosmic/670)+1

#print 'need',num_cosmic,'cosmic'
#print 'need',num_bnb,'bnb'

# 4 hours
cosmic_stop = 5247792
bnb_stop = 108*40

# 2 hours
cosmic_stop = int(cosmic_stop/2.)+1
bnb_stop = int(bnb_stop/2.)+1

# 8 hours
#bnb_stop *=2

# 24 hours
#bnb_stop *= 6

df_cosmic = open_df('flash_trigger_ana/cosmic.root',start=0,stop=cosmic_stop)
df_bnb = open_df('flash_trigger_ana/bnb.root',start=0,stop=bnb_stop)



In [ ]:
print len(df_cosmic.groupby(['run','subrun','event'])),len(df_cosmic)
print len(df_bnb.groupby(['run','subrun','event'])),len(df_cosmic)

In [ ]:
#
# Flash Timing distribution 
#
cosmic_cand = df_cosmic.query('pe_total>50')
bnb_cand = df_bnb.query('pe_total>50')
fig, ax = plt.subplots(figsize=(12, 8))
cosmic_cand['dt'].hist(ax=ax,bins=200,color='blue')
bnb_cand['dt'].hist(ax=ax,bins=200,color='red')
plt.xlabel('Time w.r.t. Trigger [us]',fontsize=20)
plt.ylabel('Flash Count',fontsize=20)
plt.title('Flash Time Distribution',fontsize=20)
ax.tick_params(labelsize=20)
plt.show()

cosmic_dt = cosmic_cand['dt'].ravel()
bnb_dt = bnb_cand['dt'].ravel()

kazu=[]
for x in cosmic_dt:
    x += 1600.
#    if x > 2400: x -= 2400
#    while x > 2400.:
#        x -= 2400.
    x -= 400
    kazu.append(x)

andy=[]
for x in bnb_dt:
    andy.append(x)

cosmic_spec, cosmic_bins = np.histogram(kazu,bins=500,range=(-400,400))
bnb_spec, bnb_bins = np.histogram(andy,bins=500,range=(-400,400))
cosmic_err=[]
for x in cosmic_spec:
    cosmic_err = np.sqrt(x)
    
bnb_err=[]
for x in cosmic_spec:
    bnb_err = np.sqrt(x)
    
added_spec=[]
added_err=[]
ymax=0
for x in xrange(len(bnb_spec)):
    if x < 260 and x > 240:
        print x, bnb_spec[x],cosmic_spec[x]
    added_spec.append(bnb_spec[x]+cosmic_spec[x])
    added_err = np.sqrt(added_spec[-1])
    if ymax < added_spec[-1]: 
        ymax=added_spec[-1]

plt.figure(figsize=(12,8))
ymin = 0
ybuf = (ymax - ymin) * 0.05
ymax += ybuf
ymin -= ybuf
plt.figure(figsize=(12,8))
plt.ylim(ymin,ymax)
plt.xlim(-400,400)
plt.errorbar(np.arange(-400,400,1.6), added_spec, yerr=added_err,marker='o', linestyle='',color='red')
plt.xlabel('Time w.r.t. Trigger [us]',fontsize=20)
plt.ylabel('Flash Count',fontsize=20)
plt.tick_params(labelsize=20)
plt.grid()
plt.title('OpFlash (> 50 P.E.) Time Distribution',fontsize=20)
plt.show()
